In [ ]:
# import statements
from ultralytics import YOLO  #object detection algorithm
import cv2  #videos
import pandas as pd #csv
import os #to get files automatically

In [ ]:
# Get participant and recording number
participant_number = input("Enter participant number: ")
recording_number = input("Enter recording number: ") 

base_path = "/net/store/nbp/projects/OsnaPlaza/Body/raw/unprocessed videos"
saving_file_path = "/net/store/nbp/projects/OsnaPlaza/Body/raw"

### Annotating video and making a dataframe with detections ###

In [ ]:
# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# input video
video_path = os.path.join(base_path, f"{participant_number}_{recording_number}.mp4")
cap = cv2.VideoCapture(video_path)

# input video information
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames
fr = cap.get(cv2.CAP_PROP_FPS)  # Frame rate (frames per second)
video_duration = frame_count / fr  # Duration in seconds

print(f"Total frames: {frame_count}")
print(f"Frame rate: {fr} FPS")
print(f"Video duration: {video_duration:.2f} seconds")

# for output video (annotated video)
output_path = os.path.join(base_path, f"yolo_{participant_number}_{recording_number}.mp4")
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fr, (int(cap.get(3)), int(cap.get(4)))) #making sure the output video has same attributes as the input 

# comment out previous line if don't want the video output to be saved

## for a detections df
detections = [] # a list to for detection results
frame_count = 0 # initialising frame count

while cap.isOpened(): # a loop getting frames until the video is finished
    ret, frame = cap.read() 
    if not ret:
        break

    frame_count += 1  # Track the frame number

    # Run YOLOv8 detection on the current frame
    results = model(frame,classes=0) # results = model(frame,classes=0) - for only person detection

    # Save detections to the list
    for result in results[0].boxes.data:
        x_min, y_min, x_max, y_max, confidence, class_id = result
        class_label = model.names[int(class_id)]  # get class name from class ID
        detections.append([frame_count, x_min.item(), y_min.item(), x_max.item(), y_max.item(), confidence.item(), class_label])

    # Annotate frame with bounding boxes and save to output video
    annotated_frame = results[0].plot()
    out.write(annotated_frame)

    ##  to display the annotated frame during the process
    '''cv2.imshow("YOLO Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break'''

# making a df
columns = ["Frame", "X_min", "Y_min", "X_max", "Y_max", "Confidence", "Class"]
detections_df = pd.DataFrame(detections, columns=columns)
saving_path = os.path.join(saving_file_path, f"detections_{participant_number}_{recording_number}.csv")
detections_df.to_csv(saving_path, index=False)

cap.release() # closes the video file
out.release() # closes the video writer
cv2.destroyAllWindows() # closes any OpenCV windows

display(detections_df)